In [1]:
from vnpy_portfoliostrategy.mysqlservice import MysqlService

In [2]:
from datetime import datetime
import pandas as pd
import numpy as np

In [3]:
db = MysqlService()

current_day = datetime(2024,9,20)
rebal_tar = db.select('daily_rebalance_target',today = current_day, strategy = 'strategy2')
trading_schedule = db.select('trading_schedule',today = current_day, strategy = 'strategy2').set_index('id').drop_duplicates()
previous_trading_schedule = db.select('trading_schedule',date = current_day, strategy = 'strategy2').set_index('id').drop_duplicates()
trading_hours = db.select('trading_hours',date = trading_schedule['date'].iloc[0])
pos_data = db.select('current_pos', strategy = 'strategy2')['symbol'].values.tolist()

if trading_schedule.shape[0]!=1 or previous_trading_schedule.shape[0]!=1:
    raise Exception('Wrong trading schedule for strategy2')

rebal_tar = pd.concat([pd.Series(rebal_tar['symbol'].values[0].split(',')),
                        pd.Series(rebal_tar['target'].values[0].split(','))],axis=1,keys=['symbol','target'])
to_trade_df = pd.concat([pd.Series(trading_schedule['symbol'].values[0].split(',')).str[:-4],
                            pd.Series(trading_schedule['symbol'].values[0].split(','))],axis=1,keys=['symbol','symb']
                        ).merge(trading_hours[['rqsymbol','symbol']],left_on='symb',right_on='rqsymbol',how='inner'
                                ).merge(rebal_tar,left_on='symbol_x',right_on='symbol',how='inner'
                                        )[['symbol_y','target']]
                                
db.close()


In [ ]:
pd.DataFrame(pos_data[5:]).reset_index()

In [5]:
pd.DataFrame(pos_data[5:]).join(to_trade_df,how='outer')

,0,symbol_y,target
0,c2411.DCE,a2501.DCE,1
1,cs2411.DCE,al2411.SHFE,-1
2,eb2411.DCE,ao2411.SHFE,0
3,fu2411.SHFE,AP501.CZCE,-1
4,i2501.DCE,b2411.DCE,-1
5,jd2501.DCE,br2411.SHFE,-1
6,lh2411.DCE,bu2411.SHFE,1
7,m2501.DCE,c2411.DCE,0
8,p2501.DCE,CF501.CZCE,0
9,pg2411.DCE,CJ501.CZCE,1


In [ ]:
pd.DataFrame(pos_data[5:]).set_index(0)

In [11]:
pd.concat([pd.Series(trading_schedule['symbol'].values[0].split(',')).str[:-4],
                            pd.Series(trading_schedule['symbol'].values[0].split(','))],axis=1,keys=['symbol','symb']
                        ).merge(trading_hours[['rqsymbol','symbol']],left_on='symb',right_on='rqsymbol',how='inner'
                                )

,symbol_x,symb,rqsymbol,symbol_y
0,A,A2501,A2501,a2501.DCE
1,AL,AL2411,AL2411,al2411.SHFE
2,AO,AO2411,AO2411,ao2411.SHFE
3,AP,AP2501,AP2501,AP501.CZCE
4,B,B2411,B2411,b2411.DCE
...,...,...,...,...
62,WH,WH2411,WH2411,WH411.CZCE
63,WR,WR2410,WR2410,wr2410.SHFE
64,Y,Y2501,Y2501,y2501.DCE
65,ZC,ZC2410,ZC2410,ZC410.CZCE


In [7]:
rollover_df = pd.concat([pd.Series(previous_trading_schedule['symbol'].values[0].split(',')),
                 pd.Series(trading_schedule['symbol'].values[0].split(','))],axis=1,keys=['pre','post'])

In [20]:
rollover_df.query("pre!=post").replace(trading_hours[['rqsymbol','symbol']].set_index('rqsymbol').to_dict()['symbol'])

,pre,post
63,WR2409,wr2410.SHFE


In [19]:
rollover_df.query("pre!=post").map(lambda x: len(str(x)))

,pre,post
63,WR2409,WR2410


In [15]:
trading_hours[['rqsymbol','symbol']].set_index('rqsymbol').to_dict()['symbol']

{'A2501': 'a2501.DCE',
 'AG2412': 'ag2412.SHFE',
 'AL2411': 'al2411.SHFE',
 'AO2411': 'ao2411.SHFE',
 'AP2501': 'AP501.CZCE',
 'AU2412': 'au2412.SHFE',
 'B2411': 'b2411.DCE',
 'BB2501': 'bb2501.DCE',
 'BC2411': 'bc2411.INE',
 'BR2411': 'br2411.SHFE',
 'BU2411': 'bu2411.SHFE',
 'C2411': 'c2411.DCE',
 'CF2501': 'CF501.CZCE',
 'CJ2501': 'CJ501.CZCE',
 'CS2411': 'cs2411.DCE',
 'CU2410': 'cu2410.SHFE',
 'CY2501': 'CY501.CZCE',
 'EB2411': 'eb2411.DCE',
 'EC2412': 'ec2412.INE',
 'EG2501': 'eg2501.DCE',
 'FB2501': 'fb2501.DCE',
 'FG2501': 'FG501.CZCE',
 'FU2411': 'fu2411.SHFE',
 'HC2501': 'hc2501.SHFE',
 'I2501': 'i2501.DCE',
 'IC2410': 'IC2410.CFFEX',
 'IF2410': 'IF2410.CFFEX',
 'IH2410': 'IH2410.CFFEX',
 'IM2410': 'IM2410.CFFEX',
 'J2501': 'j2501.DCE',
 'JD2501': 'jd2501.DCE',
 'JM2501': 'jm2501.DCE',
 'JR2411': 'JR411.CZCE',
 'L2501': 'l2501.DCE',
 'LC2411': 'lc2411.GFEX',
 'LH2411': 'lh2411.DCE',
 'LR2411': 'LR411.CZCE',
 'LU2412': 'lu2412.INE',
 'M2501': 'm2501.DCE',
 'MA2501': 'MA501.CZC